In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite

In [28]:
import networkx as nx

In [8]:
def remove_nodes(G, low_deg, low_layer, weight, *args):
    ''' remove nodes from a networkx graph G, that have too low a degree (lower than or equal to low_deg) and
    exist in too few layers (lower than or equal to low_layer). 
    The other layers (of form of networkx graphs as well) are given in args.
    For weighted graph, can use weight = 'weight'
    '''
    need_remove = []
    for node in G.nodes():
        layers = 1
        for L in args:
            if (L.has_node(node) == True):
                layers = layers + 1
        if ((G.degree[node] <= low_deg) & (layers <= low_layer)):
            need_remove.append(node)
    print(need_remove)
    G.remove_nodes_from(need_remove)

In [9]:
def remove_non_common(G, comm_heroes):
    node_lis = list(G.nodes)
    remove_lis = []
    for i in node_lis:
        if i not in comm_heroes:
            remove_lis += [i]
            
    G.remove_nodes_from(remove_lis)
    return G

In [10]:
def load_temporal_MC_flow_graph(foldername, weighted = True, size = "common"):
    # load graph from file
    data = pd.read_csv(foldername+'/edges.csv') 
    graph = {}
    
    # temporal periods name
    graph['layer_names'] = ["silver", "bronze", "modern", "heroes"]
    graph['T'] = len(graph['layer_names'])
    
    # data cleaning
    layers = []
    for i in ["A ", "A2", "A3"]:
        filters = data["comic"].str.startswith(i)
        layers.append(data[filters])
        
    layers[0]['issue'] = layers[0]['comic'].str[2:]
    issues1 = layers[0][~layers[0]['issue'].str.startswith("'")]
    for i in range(len(issues1)):
        issues1.iloc[i, 2] = issues1.iloc[i, 2].split("/",1)[0]
        issues1.iloc[i, 2] = issues1.iloc[i, 2].split("-",1)[0]
    
    # first construct four bipartite networks
    silver = issues1[issues1['issue'].astype(float) <= 82]
    silver = silver[["hero", "comic"]]
    
    bronze = issues1[(issues1['issue'].astype(float) > 82) & (issues1['issue'].astype(float) <= 242)]
    bronze = bronze[["hero", "comic"]]
    
    modern = issues1[issues1['issue'].astype(float) > 242]
    modern = modern[["hero", "comic"]]
    modern = pd.concat([modern, layers[1]])
    
    heroes = pd.concat([layers[2], layers[0][layers[0]['issue'].str.startswith("'")]])
    
    # project the bipartite graphs onto heros
    B = nx.Graph()
    B.add_nodes_from(silver['hero'].drop_duplicates(), bipartite=0)
    B.add_nodes_from(silver['comic'].drop_duplicates(), bipartite=1)
    B.add_edges_from([(row['hero'], row['comic']) for idx, row in silver.iterrows()])
    if weighted == True:
        Gsilver_weighted = bipartite.weighted_projected_graph(B, silver['hero'].drop_duplicates())
    else: 
        Gsilver = bipartite.projected_graph(B, silver['hero'])
    
    B = nx.Graph()
    B.add_nodes_from(bronze['hero'].drop_duplicates(), bipartite=0)
    B.add_nodes_from(bronze['comic'].drop_duplicates(), bipartite=1)
    B.add_edges_from([(row['hero'], row['comic']) for idx, row in bronze.iterrows()])
    if weighted == True:
        Gbronze_weighted = bipartite.weighted_projected_graph(B, bronze['hero'].drop_duplicates())
    else:
        Gbronze = bipartite.projected_graph(B, bronze['hero'])
    
    B = nx.Graph()
    B.add_nodes_from(modern['hero'].drop_duplicates(), bipartite=0)
    B.add_nodes_from(modern['comic'].drop_duplicates(), bipartite=1)
    B.add_edges_from([(row['hero'], row['comic']) for idx, row in modern.iterrows()])
    if weighted == True:
        Gmodern_weighted = bipartite.weighted_projected_graph(B, modern['hero'].drop_duplicates())
    else:
        Gmodern = bipartite.projected_graph(B, modern['hero'])
    
    B = nx.Graph()
    B.add_nodes_from(heroes['hero'].drop_duplicates(), bipartite=0)
    B.add_nodes_from(heroes['comic'].drop_duplicates(), bipartite=1)
    B.add_edges_from([(row['hero'], row['comic']) for idx, row in heroes.iterrows()])
    if weighted == True:
        Gheroes_weighted = bipartite.weighted_projected_graph(B, heroes['hero'].drop_duplicates())
    else:
        Gheroes = bipartite.projected_graph(B, heroes['hero'])
      
    
    if weighted == True:
        networks = {'silver': Gsilver_weighted, 'bronze': Gbronze_weighted, 'modern': Gmodern_weighted, 'heroes': Gheroes_weighted}
    else:
        networks = {'silver': Gsilver, 'bronze': Gbronze, 'modern': Gmodern, 'heroes': Gheroes}
            
    # Add or remove nodes from graph depending on "size”
    if size == "common": 
        # find common heros
        comm_heroes = list(set(silver['hero']).intersection(bronze['hero']))
        comm_heroes = list(set(comm_heroes).intersection(modern['hero']))
        comm_heroes = list(set(comm_heroes).intersection(heroes['hero']))


        # a tensor containing network adjacency matrix at each time
        graph['A_tensor'] = []
        for layer in graph['layer_names']:
            network = remove_non_common(networks[layer], comm_heroes)
            network_matrix = nx.to_numpy_array(network, sorted(list(network.nodes())))
            print(sorted(list(network.nodes())))
            graph['nodenames'] = sorted(list(network.nodes()))
            graph['N'] = len(graph['nodenames'])
            graph['A_tensor'].append(network_matrix)
            
            
            
    elif size == "full":
        full_heros = list(set().union(silver['hero'], bronze['hero'], modern['hero'], heroes['hero']))
        
        # a tensor containing network adjacency matrix at each time
        graph['A_tensor'] = []
        for layer in graph['layer_names']:
            network = networks[layer]
            network.add_nodes_from(full_heros)
            network_matrix = nx.to_numpy_array(network, sorted(list(network.nodes())))
            print(sorted(list(network.nodes())))
            graph['nodenames'] = sorted(list(network.nodes()))
            graph['N'] = len(graph['nodenames'])
            graph['A_tensor'].append(network_matrix)
    
    return graph

In [12]:
graph = load_temporal_MC_flow_graph("edges.csv")

C:\Users\jim13\AppData\Local\Temp\ipykernel_56144\3271320431.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  layers[0]['issue'] = layers[0]['comic'].str[2:]
C:\Users\jim13\AppData\Local\Temp\ipykernel_56144\3271320431.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issues1.iloc[i, 2] = issues1.iloc[i, 2].split("/",1)[0]
C:\Users\jim13\AppData\Local\Temp\ipykernel_56144\3271320431.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

['ANT-MAN/DR. HENRY J.', 'ATTUMA', 'BEAST/HENRY &HANK& P', 'BINARY/CAROL DANVERS', 'BLACK KNIGHT V/DANE', "BLACK PANTHER/T'CHAL", 'BLACK WIDOW/NATASHA', 'CAPTAIN AMERICA', 'CAPTAIN MARVEL II/MO', 'COUNT NEFARIA, LUCHI', 'CRYSTAL [INHUMAN]', 'DAREDEVIL/MATT MURDO', 'DR. DRUID/ANTHONY LU', 'FALCON/SAM WILSON', 'GRIM REAPER/ERIC WIL', 'GYRICH, HENRY PETER', 'HAWK', 'HELLCAT/PATSY WALKER', 'HERCULES [GREEK GOD]', 'HULK/DR. ROBERT BRUC', 'HUMAN TORCH/JOHNNY S', 'ICEMAN/ROBERT BOBBY', 'INVISIBLE WOMAN/SUE', 'IRON MAN/TONY STARK', 'JARVIS, EDWIN', 'LUNA/LUNA MAXIMOFF [', 'MOCKINGBIRD/DR. BARB', 'MOONDRAGON/HEATHER D', 'MR. FANTASTIC/REED R', "O'BRIEN, MICHAEL", 'PHARAOH RAMA-TUT', 'POWER MAN/ERIK JOSTE', 'QUICKSILVER/PIETRO M', 'SCARLET WITCH/WANDA', 'SHE-HULK/JENNIFER WA', 'SPIDER-MAN/PETER PARKER', 'STARFOX/EROS', 'SUB-MARINER/NAMOR MA', 'SWORDSMAN/JACQUES DU', 'THING/BENJAMIN J. GR', 'THOR/DR. DONALD BLAK', 'TIGRA/GREER NELSON', 'ULTRON', 'VISION', 'WASP/JANET VAN DYNE', 'WIZARD/BENTLEY WI

In [19]:
graph

{'layer_names': ['silver', 'bronze', 'modern', 'heroes'],
 'T': 4,
 'A_tensor': [array([[ 0.,  2., 11., ..., 66.,  0.,  9.],
         [ 2.,  0.,  2., ...,  2.,  0.,  1.],
         [11.,  2.,  0., ..., 11.,  0.,  5.],
         ...,
         [66.,  2., 11., ...,  0.,  1.,  8.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 9.,  1.,  5., ...,  8.,  0.,  0.]]),
  array([[ 0.,  3., 42., ..., 62.,  1., 39.],
         [ 3.,  0.,  3., ...,  3.,  0.,  3.],
         [42.,  3.,  0., ..., 58.,  0., 51.],
         ...,
         [62.,  3., 58., ...,  0.,  3., 51.],
         [ 1.,  0.,  0., ...,  3.,  0.,  0.],
         [39.,  3., 51., ..., 51.,  0.,  0.]]),
  array([[ 0.,  0.,  5., ..., 32.,  2.,  9.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 5.,  0.,  0., ...,  1.,  0.,  1.],
         ...,
         [32.,  1.,  1., ...,  0.,  1.,  8.],
         [ 2.,  0.,  0., ...,  1.,  0.,  1.],
         [ 9.,  0.,  1., ...,  8.,  1.,  0.]]),
  array([[ 0.,  0.,  6., ..., 25.,  0., 23.],

## For the silver layer

In [22]:
nodes = graph["nodenames"]
silver = graph["A_tensor"][0]

In [23]:
silver

array([[ 0.,  2., 11., ..., 66.,  0.,  9.],
       [ 2.,  0.,  2., ...,  2.,  0.,  1.],
       [11.,  2.,  0., ..., 11.,  0.,  5.],
       ...,
       [66.,  2., 11., ...,  0.,  1.,  8.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 9.,  1.,  5., ...,  8.,  0.,  0.]])

In [67]:
cent_dfs = pd.DataFrame(index=nodes)
gs = nx.from_numpy_matrix(silver)

cent_ = nx.eigenvector_centrality(gs)
cent_df['eigenvalue_cent'] = list(cent_.values())

cent_ = nx.pagerank(gs)
cent_df['pagerank_cent'] = list(cent_.values())

cent_ = nx.closeness_centrality(gs)
cent_df['closeness_cent'] = list(cent_.values())

cent_ = nx.betweenness_centrality(gs)
cent_df['between_cent'] = list(cent_.values())

cent_ = nx.degree_centrality(gs)
cent_df['degree_cent'] = list(cent_.values())
cent_df

,eigenvalue_cent,pagerank_cent,closeness_cent,between_cent,degree_cent
ANT-MAN/DR. HENRY J.,0.181483,0.065034,0.920000,0.016086,0.913043
ATTUMA,0.129113,0.008039,0.696970,0.000640,0.565217
BEAST/HENRY &HANK& P,0.162609,0.019677,0.821429,0.007926,0.782609
BINARY/CAROL DANVERS,0.073402,0.005927,0.589744,0.000000,0.304348
BLACK KNIGHT V/DANE,0.160959,0.017828,0.807018,0.005918,0.760870
BLACK PANTHER/T'CHAL,0.162902,0.030933,0.821429,0.008001,0.782609
BLACK WIDOW/NATASHA,0.162560,0.028001,0.836364,0.013439,0.804348
CAPTAIN AMERICA,0.188929,0.065987,1.000000,0.030231,1.000000
CAPTAIN MARVEL II/MO,0.173685,0.018809,0.884615,0.016089,0.869565
"COUNT NEFARIA, LUCHI",0.072819,0.004758,0.589744,0.000000,0.304348


In [76]:
cent_df = cent_df.sort_values(by=['eigenvalue_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['eigenvalue_cent'],
)])
fig.update_layout(title_text='<b>Eigenvalue Centrality of Heros</b>')

In [77]:
cent_df = cent_df.sort_values(by=['pagerank_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['pagerank_cent'],
)])
fig.update_layout(title_text='<b>Pagerank Centrality of Heros</b>')

In [78]:
cent_df = cent_df.sort_values(by=['closeness_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['closeness_cent'],
)])
fig.update_layout(title_text='<b>Closeness Centrality of Heros</b>')

In [79]:
cent_df = cent_df.sort_values(by=['between_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['between_cent'],
)])
fig.update_layout(title_text='<b>Between Centrality of Heros</b>')

In [80]:
cent_df = cent_df.sort_values(by=['degree_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['degree_cent'],
)])
fig.update_layout(title_text='<b>Degree Centrality of Heros</b>')

## For the bronze layer

In [82]:
bronze = graph["A_tensor"][1]

In [83]:
cent_df = pd.DataFrame(index=nodes)
gs = nx.from_numpy_matrix(bronze)

cent_ = nx.eigenvector_centrality(gs)
cent_df['eigenvalue_cent'] = list(cent_.values())

cent_ = nx.pagerank(gs)
cent_df['pagerank_cent'] = list(cent_.values())

cent_ = nx.closeness_centrality(gs)
cent_df['closeness_cent'] = list(cent_.values())

cent_ = nx.betweenness_centrality(gs)
cent_df['between_cent'] = list(cent_.values())

cent_ = nx.degree_centrality(gs)
cent_df['degree_cent'] = list(cent_.values())
cent_df

,eigenvalue_cent,pagerank_cent,closeness_cent,between_cent,degree_cent
ANT-MAN/DR. HENRY J.,0.199199,0.043805,0.920000,0.025286,0.913043
ATTUMA,0.064404,0.004792,0.560976,0.000000,0.217391
BEAST/HENRY &HANK& P,0.197452,0.043777,0.901961,0.022022,0.891304
BINARY/CAROL DANVERS,0.124048,0.017955,0.657143,0.001313,0.478261
BLACK KNIGHT V/DANE,0.112997,0.006864,0.638889,0.002525,0.434783
BLACK PANTHER/T'CHAL,0.174121,0.030879,0.807018,0.012904,0.760870
BLACK WIDOW/NATASHA,0.146474,0.013810,0.707692,0.003789,0.586957
CAPTAIN AMERICA,0.208912,0.074530,1.000000,0.042770,1.000000
CAPTAIN MARVEL II/MO,0.127612,0.013014,0.676471,0.004077,0.521739
"COUNT NEFARIA, LUCHI",0.080624,0.005269,0.582278,0.000000,0.282609


In [84]:
cent_df = cent_df.sort_values(by=['eigenvalue_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['eigenvalue_cent'],
)])
fig.update_layout(title_text='<b>Eigenvalue Centrality of Heros</b>')

In [85]:
cent_df = cent_df.sort_values(by=['pagerank_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['pagerank_cent'],
)])
fig.update_layout(title_text='<b>Pagerank Centrality of Heros</b>')

In [86]:
cent_df = cent_df.sort_values(by=['closeness_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['closeness_cent'],
)])
fig.update_layout(title_text='<b>Closeness Centrality of Heros</b>')

In [87]:
cent_df = cent_df.sort_values(by=['between_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['between_cent'],
)])
fig.update_layout(title_text='<b>Between Centrality of Heros</b>')

In [88]:
cent_df = cent_df.sort_values(by=['degree_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['degree_cent'],
)])
fig.update_layout(title_text='<b>Degree Centrality of Heros</b>')

## For the modern layer

In [89]:
modern = graph["A_tensor"][2]

In [90]:
cent_df = pd.DataFrame(index=nodes)
gs = nx.from_numpy_matrix(modern)

cent_ = nx.eigenvector_centrality(gs)
cent_df['eigenvalue_cent'] = list(cent_.values())

cent_ = nx.pagerank(gs)
cent_df['pagerank_cent'] = list(cent_.values())

cent_ = nx.closeness_centrality(gs)
cent_df['closeness_cent'] = list(cent_.values())

cent_ = nx.betweenness_centrality(gs)
cent_df['between_cent'] = list(cent_.values())

cent_ = nx.degree_centrality(gs)
cent_df['degree_cent'] = list(cent_.values())
cent_df

,eigenvalue_cent,pagerank_cent,closeness_cent,between_cent,degree_cent
ANT-MAN/DR. HENRY J.,0.185835,0.039568,0.920000,0.019594,0.913043
ATTUMA,0.032393,0.003646,0.534884,0.000000,0.130435
BEAST/HENRY &HANK& P,0.138824,0.008528,0.707692,0.000351,0.586957
BINARY/CAROL DANVERS,0.040093,0.004869,0.547619,0.000000,0.173913
BLACK KNIGHT V/DANE,0.171203,0.046643,0.867925,0.033906,0.847826
BLACK PANTHER/T'CHAL,0.158802,0.015319,0.779661,0.005092,0.717391
BLACK WIDOW/NATASHA,0.181141,0.037955,0.884615,0.016532,0.869565
CAPTAIN AMERICA,0.186788,0.069760,0.938776,0.027146,0.934783
CAPTAIN MARVEL II/MO,0.179031,0.040651,0.867925,0.014534,0.847826
"COUNT NEFARIA, LUCHI",0.021320,0.004400,0.528736,0.000000,0.108696


In [91]:
cent_df = cent_df.sort_values(by=['eigenvalue_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['eigenvalue_cent'],
)])
fig.update_layout(title_text='<b>Eigenvalue Centrality of Heros</b>')

In [92]:
cent_df = cent_df.sort_values(by=['pagerank_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['pagerank_cent'],
)])
fig.update_layout(title_text='<b>Pagerank Centrality of Heros</b>')

In [93]:
cent_df = cent_df.sort_values(by=['closeness_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['closeness_cent'],
)])
fig.update_layout(title_text='<b>Closeness Centrality of Heros</b>')

In [94]:
cent_df = cent_df.sort_values(by=['between_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['between_cent'],
)])
fig.update_layout(title_text='<b>Between Centrality of Heros</b>')

In [95]:
cent_df = cent_df.sort_values(by=['degree_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['degree_cent'],
)])
fig.update_layout(title_text='<b>Degree Centrality of Heros</b>')

## For the heroes layer

In [96]:
heroes = graph["A_tensor"][3]

In [97]:
cent_df = pd.DataFrame(index=nodes)
gs = nx.from_numpy_matrix(heroes)

cent_ = nx.eigenvector_centrality(gs)
cent_df['eigenvalue_cent'] = list(cent_.values())

cent_ = nx.pagerank(gs)
cent_df['pagerank_cent'] = list(cent_.values())

cent_ = nx.closeness_centrality(gs)
cent_df['closeness_cent'] = list(cent_.values())

cent_ = nx.betweenness_centrality(gs)
cent_df['between_cent'] = list(cent_.values())

cent_ = nx.degree_centrality(gs)
cent_df['degree_cent'] = list(cent_.values())
cent_df

,eigenvalue_cent,pagerank_cent,closeness_cent,between_cent,degree_cent
ANT-MAN/DR. HENRY J.,0.179764,0.043709,0.920000,0.015426,0.913043
ATTUMA,0.070714,0.006015,0.597403,0.000069,0.326087
BEAST/HENRY &HANK& P,0.153403,0.016438,0.766667,0.001240,0.695652
BINARY/CAROL DANVERS,0.165575,0.040316,0.851852,0.011619,0.826087
BLACK KNIGHT V/DANE,0.153062,0.024963,0.779661,0.003533,0.717391
BLACK PANTHER/T'CHAL,0.159982,0.022238,0.807018,0.005291,0.760870
BLACK WIDOW/NATASHA,0.167607,0.020166,0.867925,0.014043,0.847826
CAPTAIN AMERICA,0.187148,0.055033,1.000000,0.027939,1.000000
CAPTAIN MARVEL II/MO,0.150902,0.020523,0.766667,0.002379,0.695652
"COUNT NEFARIA, LUCHI",0.068298,0.007239,0.589744,0.000000,0.304348


In [98]:
cent_df = cent_df.sort_values(by=['eigenvalue_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['eigenvalue_cent'],
)])
fig.update_layout(title_text='<b>Eigenvalue Centrality of Heros</b>')

In [99]:
cent_df = cent_df.sort_values(by=['pagerank_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['pagerank_cent'],
)])
fig.update_layout(title_text='<b>Pagerank Centrality of Heros</b>')

In [100]:
cent_df = cent_df.sort_values(by=['closeness_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['closeness_cent'],
)])
fig.update_layout(title_text='<b>Closeness Centrality of Heros</b>')

In [101]:
cent_df = cent_df.sort_values(by=['between_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['between_cent'],
)])
fig.update_layout(title_text='<b>Between Centrality of Heros</b>')

In [102]:
cent_df = cent_df.sort_values(by=['between_cent'], ascending=False)
fig = go.Figure(data=[go.Bar(
    x=cent_df.index[0:15],
    y=cent_df['between_cent'],
)])
fig.update_layout(title_text='<b>Between Centrality of Heros</b>')